# Second step of fitting -- Using `OpenCV` to fit the patches of defects

In [196]:
# Results Path
import os
# Import packages
import re

import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

os.getcwd()

'/Users/mingrenshen/Projects/multitype-defect-detection'

## Test Fitting Algorithm Alone

In [214]:
def fitEllipse(rawPath, outPath, imgFile):
    """
    This function fits the ellipse on the cropped patches of the defects
    and save the fitting ellipse in "imgFile"+"_fitted.jpg" as the vistualization of final results
    
    rawPath :: the path of the result cropped images
    imgFile :: the file name of the small patch of the detected defects
    
    Return :
    ellipse : Typical Ouput of OpenCV's ellipse function : 
        center	Center of the ellipse.
        axes	Half of the size of the ellipse main axes.
        angle	Ellipse rotation angle in degrees.

        ( (center_rr, center_cc), (minor_axe, major_axe), angle )
    
    """
    img = cv2.imread(rawPath+imgFile)
    imgName = imgFile.rstrip(".jpg")
    #plt.imshow(img)
    #img = cv2.medianBlur(img, 25)
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #ret, thresh = cv2.threshold(gray,50,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C)
    #thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,23,2)
    # Otsu's thresholding after Gaussian filtering
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur_gray = cv2.GaussianBlur(gray,(3,3),0)
    ret3,thresh = cv2.threshold(blur_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # noise removal
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)) #np.ones((5,5),np.uint8) #
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 3)
    #closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    #opening = closing
    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=3)
    cv2.imwrite(outPath + imgName+'_sureBG.jpg', sure_bg)
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,1,5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.2*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    markers = cv2.watershed(img,markers)
    img[markers == -1] = [255,0,0]
    imgray = sure_fg #cv2.cvtColor(sure_fg,cv2.COLOR_BGR2GRAY)
    #ret,thresh = cv2.threshold(imgray,50,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C)
    #thresh = cv2.adaptiveThreshold(imgray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,23,2)
    ret3,thresh = cv2.threshold(imgray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    im,contours,hierarchy= cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    max_index = 0
    # if contour length is more than 1
    if len(contours) == 0:
        print("error in contour calculation")
    elif len(contours) == 1:
        contoursALL = contours[0]
    else:
        indexDict = dict()
        for idx,con in enumerate(contours):
            indexDict[cv2.contourArea(con)] = idx
        # print(indexDict)
        A = sorted(indexDict.keys(),reverse=True)
        # print(A)
        max_index = indexDict[A[0]]
        contoursALL = contours[indexDict[A[1]]]
        for i in range(len(contours)):
            if i != max_index and i != indexDict[A[1]] :
                #print(contours[i])
                tmp = contours[i]
                contoursALL = np.concatenate((contoursALL,tmp))
    cnt = contoursALL
    ellipse = cv2.fitEllipse(cnt)
    #print(ellipse)
    imgNN = cv2.imread(rawPath+imgFile)
    #cv2.imwrite(outPath+imgName+'_fitted11.jpg', imgNN)
    cv2.ellipse(imgNN,ellipse,(0,255,0),3)
    cv2.imwrite(outPath+imgName+'_fitted.jpg', imgNN)
    #plt.imshow(img)
    return ellipse

In [189]:
# Path of Raw Results for testing single image fitting
rawPath = "./RawResult0p1/7/"
outPath = "./outputFitting0p1/"
for f in os.listdir(rawPath):
    if f != '.DS_Store':
        print(f)
        fitEllipse(rawPath,outPath,f)

48.jpg
49.jpg
8.jpg
9.jpg
14.jpg
28.jpg
29.jpg
15.jpg
17.jpg
16.jpg
12.jpg
13.jpg
39.jpg
11.jpg
10.jpg
38.jpg
21.jpg
35.jpg
34.jpg
20.jpg
36.jpg
22.jpg
23.jpg
37.jpg
33.jpg
27.jpg
26.jpg
32.jpg
18.jpg
24.jpg
30.jpg
31.jpg
25.jpg
19.jpg
42.jpg
4.jpg
56.jpg
5.jpg
57.jpg
43.jpg
55.jpg
7.jpg
41.jpg
40.jpg
54.jpg
6.jpg
2.jpg
50.jpg
44.jpg
45.jpg
3.jpg
51.jpg
47.jpg
53.jpg
1.jpg
52.jpg
0.jpg
46.jpg


# Whole Fitting Pipeline

**Input** 
1. `pix2nm` : records down the pixel to nm information of each image
2. `rawPrediction` : the folder generated by `RawPredictionGeneration.ipynb`

In [27]:
# Generate Converstion Factor
# all following the rules that 
# 0 for 111
# 1 for black dot
# 2 for 100

# pix2nm is the dict that stores all the coefficient to convert pixel value to real nm data
# The format is { 'img_name' :  [pixelNum,nmNum]}
# in total the full name of all 12 testing images should be here
# pix2nm is the dict that stores all the coefficient to convert pixel value to real nm data
# The format is { 'img_name' :  [pixelNum,nmNum]}
# in total the full name of all 12 testing images should be here
pix2nm = {'0501_300kx_1nm_clhaadf3_0010.jpg' : [1024, 490],
          '0501_300kx_1nm_clhaadf3_0014.jpg' : [1024, 490],
          '1ROI_100kx_4100CL_foil1.jpg' : [1024, 890],
          '200kV_500kx_p2nm_8cmCL_grain1_0056 - Copy.jpg' : [1024, 290], #[1024, 291.248],
          '200kV_500kx_p2nm_8cmCL_grain2_0036.jpg' : [1024, 490],#[1024, 485.413],
          '5401_300kx_1nm_clhaadf3_0020.jpg' : [1024, 490],
          '8ROI_100kx_4100CL_foil1.jpg' : [1024, 890],
          'BF X500K, 06 (2).jpg' : [1024, 145],
          'g1_backonzone_GBtowardsfrom_0007.jpg' : [2048, 290], #[2048, 291.248],
          'g2_midonzone_GBtowardsfront_0010.jpg' : [2048, 290], #[2048, 291.248],
          'grid1_roi1_500kx_0p5nm_haadf1_0025.jpg' : [1024, 290],
          'grid1_roi2_500kx_0p5nm_haadf1_0047.jpg': [1024, 290]}

In [198]:
# OverAll Fitting for all images
rawPath = "./RawResult0p05/"
outPath = "./outputFitting0p05/"
# In Total we have 12 testing images so first read in the `log.txt` to gain the imgName to testID information
testID2imgName = {}
with open(rawPath+'log.txt') as flog:
    for line in flog.readlines():
        numID, fname = line.split(' , ')
        testID2imgName[numID.strip()] = fname.strip()
print(testID2imgName)

{'11': 'grid1_roi2_500kx_0p5nm_haadf1_0047', '10': 'grid1_roi1_500kx_0p5nm_haadf1_0025', '1': '0501_300kx_1nm_clhaadf3_0014', '0': '0501_300kx_1nm_clhaadf3_0010', '3': '200kV_500kx_p2nm_8cmCL_grain1_0056 - Copy', '2': '1ROI_100kx_4100CL_foil1', '5': '5401_300kx_1nm_clhaadf3_0020', '4': '200kV_500kx_p2nm_8cmCL_grain2_0036', '7': 'BF X500K, 06 (2)', '6': '8ROI_100kx_4100CL_foil1', '9': 'g2_midonzone_GBtowardsfront_0010', '8': 'g1_backonzone_GBtowardsfrom_0007'}


In [194]:
def getReff(ra,rb,cls):
    '''
    return the effective radius of defects based on the class, major and minor axes
    
    ra : major axes full length
    rb : minor axes full length
    cls : class of the defects
    
    return
    reff : the effective radius of defects
    
    '''
    if cls == 0 or cls == 2:
        # Reff should be half of 2a
        reff = ra * 0.5
    else:
        # this is due to reff = sqrt(2a * 2b * 0.25)
        reff = np.sqrt(0.25 * ra * rb)
    return reff

In [177]:
# read in all csv files with pandas dataframe
for imgID in range(12):
    print(imgID)
    imgRawInfo = pd.read_csv(rawPath + 'results_' + str(imgID) + '.csv')
    # check the number of defects patches matchs the number of defects recorded in csv
    totalPatchNum = 0
    for f in os.listdir(rawPath + str(imgID)):
        if f.endswith(".jpg"):
            totalPatchNum += 1
    # if number of patchs mismatchs the records in csv `assert` will report an error
    assert(totalPatchNum == imgRawInfo.shape[0])
    # create the output Path to store all the results
    try:
        os.mkdir(str(outPath + str(imgID)))
    except OSError:
        print("Creation of the directory is failed")

0
1
2
3
4
5
6
7
8
9
10
11


In [178]:
def getPatchRatio(rawPath, imgFile):
    """
    This function fits the size of the patch of defects
    
    rawPath :: the path of the result cropped images
    imgFile :: the file name of the small patch of the detected defects
    
    Return :
    patch_size : (width_x, hight_y)
    
    """
    img = cv2.imread(rawPath+imgFile)
    height, width, channels = img.shape
    return (width,height)

In [215]:
'''
This part of code will 
    * calculate all the needed information of the fitted ellipse
    * store them in a new csv

Typical Ouput of OpenCV's ellipse function : 
    center	Center of the ellipse.
    axes	Half of the size of the ellipse main axes.
    angle	Ellipse rotation angle in degrees.
    
    ( (center_rr, center_cc), (minor_axe, major_axe), angle )
    
Then transform them into two types of information
(1) Geo information -- effective radius (nm)
(2) Population information -- area density (# * m^-2)

'''
# Then Fitted Each Patch into Ellipse
# Path of Raw Results for testing single image fitting
# Using another loop to do fitting
for imgID in range(12):
    print(imgID)
    imgRawInfo = pd.read_csv(rawPath + 'results_' + str(imgID) + '.csv')
    totalPathchNum = imgRawInfo.shape[0]
    # Pixel to nm factor
    scaleFactor = pix2nm[testID2imgName[str(imgID)]+".jpg"]
    #print(scaleFactor)
    rawFolder = rawPath + str(imgID)+"/"
    outFolder = outPath + str(imgID)+"/"
    
    for index,row in imgRawInfo.iterrows():
        defect_patch_name = str(index)+".jpg"
        ellipseInfo = fitEllipse(rawFolder,outFolder,defect_patch_name)
        patchInfo = getPatchRatio(rawFolder,defect_patch_name)
        # (width,height) for pathchInfo
        patchConvertToOringialRatio = ( (imgRawInfo.loc[index,'Xmax'] - imgRawInfo.loc[index,'Xmin'])/(1.0*patchInfo[0]),
                                         (imgRawInfo.loc[index,'Ymax'] - imgRawInfo.loc[index,'Ymin'])/(1.0*patchInfo[1]))
        #print(patchConvertToOringialRatio)
        avgRatio = 0.5 * (patchConvertToOringialRatio[0] + patchConvertToOringialRatio[1])
        # center_x
        imgRawInfo.loc[index,'center_x'] = ellipseInfo[0][0] * avgRatio
        # center_y
        imgRawInfo.loc[index,'center_y'] = ellipseInfo[0][1] * avgRatio
        # center_x_real
        imgRawInfo.loc[index,'center_x_real'] = imgRawInfo.loc[index,'center_x']  + imgRawInfo.loc[index,'Xmin']
        # center_y_real
        imgRawInfo.loc[index,'center_y_real'] = imgRawInfo.loc[index,'center_y'] + imgRawInfo.loc[index,'Ymin']
        # minor axes
        imgRawInfo.loc[index,'minor_axe'] = ellipseInfo[1][0] * avgRatio
        # major axes
        imgRawInfo.loc[index,'major_axe'] = ellipseInfo[1][1] * avgRatio
        # Reff_pixel
        imgRawInfo.loc[index,'Reff_pixel'] = getReff(imgRawInfo.loc[index,'major_axe'], \
                                                     imgRawInfo.loc[index,'minor_axe'], \
                                                     imgRawInfo.loc[index,'class'])
        # Reff_nm
        imgRawInfo.loc[index,'Reff_nm'] = imgRawInfo.loc[index,'Reff_pixel'] * scaleFactor[1] / scaleFactor[0]
        # RotAngle_degree
        imgRawInfo.loc[index,'RotAngle_degree'] = ellipseInfo[2]
        # RotAngle_radian
        imgRawInfo.loc[index,'RotAngle_radian'] = ellipseInfo[2] * np.pi /180.0
    imgRawInfo.to_csv (outPath + 'results_' + str(imgID) + '.csv', header=True)

0
1
2
3
4
5
6
7
8
9
10
11


# The End